<a href="https://colab.research.google.com/github/Dolappo/eng_to_hausa_mlt/blob/main/ML_Translation_from_Hausa_to_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets evaluate torch pandas numpy tqdm accelerate

In [ ]:
import os
import torch
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, Dataset
import evaluate
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gc
from tqdm import tqdm

In [ ]:
gc.enable()

In [ ]:
try:
    import torch_xla.core.xla_model as xm
    IS_TPU_AVAILABLE = True
except ImportError:
    IS_TPU_AVAILABLE = False

if IS_TPU_AVAILABLE:
    device = xm.xla_device()
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")
print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB") if torch.cuda.is_available() else None


Using device: cpu


In [ ]:
def load_model_and_tokenizer(model_name="Helsinki-NLP/opus-mt-ha-en"):

    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    return model, tokenizer

In [ ]:

def prepare_data(file_path, max_samples=None):

    if file_path.startswith('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')

    data = pd.read_csv(file_path)
    if max_samples:
        data = data.sample(n=min(max_samples, len(data)), random_state=42)

    # Clean data
    data['HAUSA'] = data['HAUSA'].astype(str).str.strip()
    data['ENGLISH'] = data['ENGLISH'].astype(str).str.strip()

    # Remove very long sequences
    data = data[
        (data['HAUSA'].str.len() <= 512) &
        (data['ENGLISH'].str.len() <= 512)
    ]

    return train_test_split(data, test_size=0.2, random_state=42)


In [ ]:
def preprocess_function(examples, tokenizer, max_length=128):
    inputs = [str(text) for text in examples['HAUSA']]
    targets = [str(text) for text in examples['ENGLISH']]

    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )


    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_length,
            truncation=True,
            padding="max_length"
        )


    labels_with_ignore_index = []
    for label in labels["input_ids"]:

        labels_with_ignore_index.append(
            [label_id if label_id != tokenizer.pad_token_id else -100 for label_id in label]
        )

    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs


In [ ]:
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    decoded_preds = [pred.strip() for pred in tokenizer.batch_decode(predictions, skip_special_tokens=True)]

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = [[label.strip()] for label in tokenizer.batch_decode(labels, skip_special_tokens=True)]

    metrics = {}

    bleu = evaluate.load("bleu")
    metrics.update(bleu.compute(predictions=decoded_preds, references=decoded_labels))

    # chrf = evaluate.load("chrf")
    # metrics.update(chrf.compute(predictions=decoded_preds, references=decoded_labels))

    return metrics

In [ ]:
def main():

    if torch.cuda.is_available():
        torch.cuda.empty_cache()


    model, tokenizer = load_model_and_tokenizer()
    model = model.to(device)

    train_data, test_data = prepare_data('Eng_Hausa.csv')

    train_dataset = Dataset.from_pandas(train_data)
    test_dataset = Dataset.from_pandas(test_data)

    tokenized_train = train_dataset.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        batch_size=64,
        remove_columns=train_dataset.column_names
    )

    tokenized_test = test_dataset.map(
        lambda x: preprocess_function(x, tokenizer),
        batched=True,
        batch_size=64,
        remove_columns=test_dataset.column_names
    )

    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        learning_rate=3e-5,
        warmup_ratio=0.1,
        weight_decay=0.01,
        fp16=torch.cuda.is_available(),  # Use mixed precision only if GPU available
        gradient_accumulation_steps=2,
        logging_dir="./logs",
        logging_steps=100,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        greater_is_better=True,
        push_to_hub=False,
        gradient_checkpointing=False,
        optim="adamw_torch",
    )


    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # Train model
    try:
        trainer.train()

        # Save the final model
        trainer.save_model("./final_model")

        if os.path.exists('/content/drive'):
            save_path = '/content/drive/MyDrive/translation_model'
            trainer.save_model(save_path)
            print(f"Model saved to Google Drive at: {save_path}")

        print("Training completed successfully!")

    except Exception as e:
        print(f"An error occurred during training: {str(e)}")

    finally:

        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/3824 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/957 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-5b21ae4f9360>:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


KeyboardInterrupt: 